In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pvlib
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
raw_data = pickle.load(open("./raw/generation.pickle", "rb"))
raw_data = pd.DataFrame({"Datetime": raw_data.times, "AC": raw_data.ac})

In [4]:
data = raw_data[['Datetime', 'AC']].copy()
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)

data.rename(columns={"AC": "energy"}, inplace=True)

data["energy"] /= 10 # W to kW  and 100fold the generation

In [5]:
data_minutely = data.resample('min').interpolate(method='linear')
data_hourly = data

In [8]:
for data, resolution in zip([data_minutely, data_hourly], ["minutely", "hourly"]):
    print(f"{resolution} Max: ", np.max(data['energy']))
    print(f"{resolution} Min: ", np.min(data['energy']))

    episodes = data.groupby(pd.Grouper(freq='W'))
    
    required_length = 10000 if resolution == "minutely" else 165
    
    episodes = pd.concat([group.fillna(0) for name, group in episodes if
                                       len(group.dropna()) >= required_length and group.index[0] >= datetime(2007, 1, 1) and
                                       group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))
    
    print(f"{resolution} episodes: ", len(episodes))
    demand = pd.HDFStore(f'./{resolution}/household_energy_demand.h5')
    with pd.HDFStore(f'./{resolution}/rooftop_solar_array.h5') as store:
        i = 0
        for name, group in episodes:
            if name.date() == demand[f"eps_{i}"].index[-1].date() and not group.empty:
                store[f'eps_{i}'] = group
                i += 1
        print(len(store))
    print(f"{resolution} episodes: ", i)
    file = pd.HDFStore(f'./{resolution}/rooftop_solar_array.h5')
    print(f"Equal length: {len(file) == len(demand)}")
    file.close()
    demand.close()

minutely Max:  44.00727161894718
minutely Min:  -0.42697428328038
minutely episodes:  104
0
101
minutely episodes:  101
Equal length: True
hourly Max:  44.00727161894718
hourly Min:  -0.42697428328038
hourly episodes:  104
0
102
hourly episodes:  102
Equal length: True
